# JUICE SPICE Kernel Dataset Status
SKD version: {skd_version} <br>
Creation: {current_time} by ESA SPICE Service (ESAC/ESA). <br>


## Startup
Prepare the testing environment at first stage, so if any validation step fails stopping the execution, the tests are already defined and in fail state.

In [1]:
from spival.classes.history import TestHistory

test_history = TestHistory()
test_history.add_test('XM-C1', 'Metakernel is valid', 'Consistency')
test_history.add_test('XM-C2', 'Frame chain', 'Consistency')
test_history.add_test('XM-C3', 'Rotation matrices', 'Consistency')
test_history.add_test('XM-C4', 'Fields of view', 'Consistency')
test_history.add_test('XM-V1', 'Time deviation', 'Validity/Error', threshold='50ms')
test_history.add_test('XM-V1-BIS', 'Time correlation', 'Validity/Error', threshold='500ms')
test_history.add_test('XM-Q2', 'S/C Clock Drift', 'Quality/Evolution')
test_history.add_test('XM-V3', 'Predicted Quaternions to CK error', 'Validity/Warning', threshold='5 mdeg')
test_history.add_test('XM-V4', 'Measured Quaternions to CK error', 'Validity/Warning', threshold='5 mdeg')
test_history.add_test('XM-V5', 'OEM to SPK error', 'Validity/Error', threshold='~100m')
test_history.add_test('XM-V7-JUICE', 'JUICE Solar Array (SA) Angles', 'Validity/Error', threshold='100 mdeg')
test_history.add_test('XM-Q3', 'Attitude Error', 'Quality/Data', threshold='500 mdeg')
test_history.add_test('XM-Q4', 'JUICE Orientation (quaternions w.r.. J2000)', 'Quality/Data')
test_history.add_test('XM-Q5-MPO', 'JUICE Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q1-MPO', 'JUICE SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('BC-Q2', 'JUICE Medium Gain Antenna Angles', 'Quality/Data')
test_history.add_test('BC-Q8', 'JUICE Medium Gain Antenna - Earth Angle', 'Quality/Data')
test_history.add_test('XM-V9', 'Gap report', 'Validity/Warning')
test_history.add_test('XM-V10', 'Diff of latest orbit w.r.t. previous orbit', 'Quality/Evolution')

### XM-C1 - Test Metakernel is valid 
{XM-C1_description}

In [ ]:
from spiops import spiops
import spiceypy

spiops.load('{metakernel}')               

test_history.set_test_result('XM-C1', True)

start_time = '{start_time_measured}'                                # Start time
finish_time ='{finish_time_measured}'                               # End time
resolution = {resolution}                                           # Resolution

interval = spiops.TimeWindow(start_time, finish_time, resolution=resolution) # spiops object TimeWindow generated
sun = spiops.Target('SUN', time=interval, frame='IAU_SUN')                   # spiops object Target Sun generated
juice = spiops.Observer('JUICE', time=interval, target=sun)                  # spiops object Observer JUICE generated
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### <center><span style='color:Green'> Metakernel loaded successfully.  </span></center>

### XM-C2 - Frame chain
Ensure that frame changes operations can be performed at any time (incl. instrument frames), meaning all frames are connected.

In [ ]:
plan_mk = '{metakernel}'.replace('ops', 'plan')
spiops.load(plan_mk)
frm_start_time = '{start_time_measured}'                                # Start time
frm_finish_time ='{finish_time_measured}'                               # End time
num_samples = 100

all_frames_ok = spiops.check_frame_chain(frm_start_time, frm_finish_time, num_samples, ignore_frames=['@EW@_PLAN', '@EW@_MEAS', 'JUICE_RIME'])
test_history.set_test_result('XM-C2', all_frames_ok)

### XM-C3 - Rotation matrices
Check that all TK frames defined with matrices are defined with proper rotation matrices.

In [ ]:
all_matrices_ok = spiops.check_rotation_matrices()
test_history.set_test_result('XM-C3', all_matrices_ok)


### XM-C4 - Fields of view
Check that all instruments fields of view are defined properly.

In [ ]:
all_fovs_ok = spiops.check_fovs(max_angle_deg=89.99994)
test_history.set_test_result('XM-C4', all_fovs_ok)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V1 - Time deviation
Downloads the time deviation files for a given time range and computes the time difference between the UTC timestamp of packet calculated with SPICE (1st column) and the UTC timestamp of packet calculated from SCOS2K header (3rd column)

In [ ]:
spiops.load('{metakernel}')
max_time_diff = spiops.time_deviation('JUICE', start_time, finish_time, plot_style='line', notebook=True)
test_history.set_test_result('XM-V1', (max_time_diff != None) and (max_time_diff < 50)) # 50 milliseconds
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V1-BIS - Time correlation
Downloads a telemetry file of a given CK and computes the time difference between the UTC time (1st column) and the clock string (2nd column) in milliseconds

In [ ]:
spiops.load('{metakernel}')
target_ck = '{skd_path}/ck/{measured_ck}'
max_time_diff = spiops.time_correlation('JUICE', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V1-BIS', (max_time_diff != None) and (max_time_diff < 500)) # 500 milliseconds
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-Q2 - S/C Clock Drift 

The following plot shows the drift of the S/C Clock.

In [ ]:
spiops.load('{metakernel}')
juice.Plot('clock_drift', notebook=True)
test_history.set_test_result('XM-Q2', True)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## Coverage

The coverage provided by the SPK and CK files is displayed for the Operational and the Planning meta-kernels. 

The Data type designation includes a reference to the originator of the data, the type of data and the reference period is provided. This is a three letter acronym. The first letter defines the file originator:
*    s: Science Operation
*    m: Mission Analysis
*    f: Flight Dynamic

the second letter defines the reference period of the data:
*    c: Cruise phase reference
*    l: Science phase Long term reference
*    m: Science phase Medium term reference
*    s: Science phase Short term reference
*    o: Undefined reference

the third letter indicates the type of data
*    <span style='color:Orange'> p: Predicted data</span>
*    <span style='color:Green'> r: Reconstructed data</span>
*    <span style='color:Red'> t: Test data</span>
*    <span style='color:Purple'> c: Commanded data (from Housekeeping Telemetry)</span>
*    <span style='color:Blue'> m: Measured data (from Housekeeping Telemetry)</span>


In [ ]:
spiops.ck_coverage_timeline('{metakernel}', ['JUICE_SPACECRAFT_PLAN', 'JUICE_SPACECRAFT_MEAS'])
spiops.ck_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['JUICE_SPACECRAFT_PLAN', 'JUICE_SPACECRAFT_MEAS'])
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

In [ ]:
spiops.spk_coverage_timeline('{metakernel}', ['JUICE'])
spiops.spk_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['JUICE'])
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V3 - Predicted Quaternions to CK error
Comparison of spacecraft orientation (quaternion) between source AEM Quaternions and generated CK

In [ ]:
spiops.load('{metakernel}'.replace('ops', 'plan'))
target_ck = '{skd_path}/ck/{predicted_ck}'
max_err = spiops.ckVsAEM('JUICE', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V3', (max_err != None) and (max_err < 5)) # 5mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V4 - Measured Quaternions to CK error
Comparison of spacecraft orientation (quaternion) between source AOCS Measured Quaternions and generated CK 

In [ ]:
spiops.load('{metakernel}')
target_ck = '{skd_path}/ck/{measured_ck}'
max_err = spiops.ckVsAocs('JUICE', target_ck, plot_style='circle', notebook=True)
test_history.set_test_result('XM-V4', (max_err != None) and (max_err < 5)) # 5mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V5 - OEM to SPK error
Comparison of spacecraft position and velocity between source OEM and generated SPK 

In [ ]:
spiops.load('{metakernel}')
target_spk = '{skd_path}/spk/{reconstructed_spk}'
max_pos_err, max_vel_err = spiops.spkVsOem('JUICE', target_spk, plot_style='line', notebook=True)
test_history.set_test_result('XM-V5', (max_pos_err != None) and (max_pos_err < 0.1))
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V7-JUICE - JUICE Solar Array (SA) Angles
Comparison of JUICE SADM Angle from HK TM and SPICE CK

In [ ]:
spiops.load('{metakernel}')
max_err = spiops.saa_vs_hk_sa_position('JUICE', plot_style='circle', notebook=True)
test_history.set_test_result('XM-V7-JUICE', (max_err != None) and (max_err < 100)) # 100mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-Q3 - Attitude Error
Comparison of +Z axis orientation between predicted and measured attitude in arcseconds 

In [ ]:
spiops.load('{metakernel}')
measured_ck = '{skd_path}/ck/{measured_ck}'
predicted_ck = '{skd_path}/ck/{predicted_ck}'
resolution = 30

res = spiops.ckdiff_error(measured_ck, predicted_ck, ['JUICE_SPACECRAFT_MEAS', 'JUICE_SPACECRAFT_PLAN'], 'J2000', resolution, 0.001, 
                    plot_style='circle', utc_start=start_time, utc_finish=finish_time, notebook=True)
test_history.set_test_result('XM-Q3', (res != None) and (res < 500)) # 500mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## S/C Structures Orientation
Quaternions, JUICE Solar Array Orientation and Solar Aspect Angle, Medium Gain Antenna Orientation and MGA boresight-Earth Angle.

### XM-Q4 - JUICE Orientation (quaternions w.r.. J2000)

In [ ]:
spiops.load('{metakernel}')
juice.Plot('quaternions', notebook=True)
test_history.set_test_result('XM-Q4', True)

### XM-Q5-MPO - JUICE Solar Array (SA) Angles 

In [ ]:
juice.Plot('sa_ang', notebook=True)
test_history.set_test_result('XM-Q5-MPO', True)

### XM-Q1-MPO - JUICE SA Solar Aspect Angle

In [ ]:
juice.Plot('saa_sa', notebook=True)
test_history.set_test_result('XM-Q1-MPO', True)

### BC-Q2 - JUICE Medium Gain Antenna Angles

In [ ]:
juice.Plot('mga_angles', notebook=True)
test_history.set_test_result('BC-Q2', True)

### BC-Q8 - JUICE Medium Gain Antenna - Earth Angle

In [ ]:
juice.Plot('mga_earth', notebook=True)
test_history.set_test_result('BC-Q8', True)

### XM-V9 - Gap Report

In [ ]:
frames = ['JUICE_SPACECRAFT_MEAS','JUICE_SA+Y_MEAS', 'JUICE_MGA_AZ_MEAS']
spiops.ck_gap_report('{metakernel}', frames)
test_history.set_test_result('XM-V9', True)

### XM-V10 - Diff of latest orbit w.r.t. previous orbit

In [ ]:
spk_expression = 'juice_orbc_??????_??????_??????_v??.bsp'
num_samples = 5000
max_position_error = spiops.spk_diff('JUICE', spk_expression, num_samples, notebook=True)
test_history.set_test_result('XM-V10', (max_position_error != None))

In [ ]:
# Unload ops MK
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

In [ ]:
# Show validation results
test_history.show_tests()